In [1]:
from ipyleaflet import Map, TileLayer, LayersControl, basemaps, basemap_to_tiles, WidgetControl, LegendControl
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
import IPython
import requests
import json

# Load information dynamically created by most recent schisto model run.
nb_json_config_path = 'http://localhost:8080/output/nb-json-config.json'

# Get the layers from the most recent model run to display
nb_json_config = requests.get(nb_json_config_path).json()
layer_name_list = []
for layer in nb_json_config['layers']:
    layer_name_list.append(layer + "_tiles")

# Get the plot png paths from the most recent model run
plot_root_path = 'http://localhost:8080/intermediate/plot_previews/'
plot_png_list = [plot_root_path + plot_name for plot_name in nb_json_config['plot_paths']]

# Get map center
map_center = nb_json_config['aoi_center']

# Get legend colors
#hab_suit_legend = [("0%", "#ffffb2"), ("25%", "#fecc5c"), ("50%", "#fd8d3c"), ("75%", "#f03b20"), ("100%", "#bd0026")]
hab_suit_colorbar = [(0, "#ffffb2"), (0.25, "#fecc5c"), (0.5, "#fd8d3c"), (0.75, "#f03b20"), (1.0, "#bd0026")]
#pop_suit_legend = [("0%", "#f7fbff"), ("20%", "#d1e2f3"), ("40%", "#9ac8e0"), ("60%", "#529dcc"), ("80%", "#1d6cb1"), ("100%", "#08306b")]
pop_suit_colorbar = [(0, "#f7fbff"), (0.20, "#d1e2f3"), (0.40, "#9ac8e0"), (0.60, "#529dcc"), (0.80, "#1d6cb1"), (1.0, "#08306b")]
#legend_hab_widget = widgets.HTML(description="hab", value="<b>hab<b>")
#legend_pop_widget = widgets.HTML(description="pop", value="<b>pop<b>")
# LEGEND CONTROL
# This method is taken from https://github.com/jupyter-widgets/ipyleaflet/issues/706
import matplotlib.pyplot as plt; import matplotlib as mpl
import io
# These hex values map to generic-risk-styles RGB
#color_list_ex = [(0, "#ffffb2"), (0.25, "#fecc5c"), (0.5, "#fd8d3c"), (0.75, "#f03b20"), (1.0, "#bd0026")]
colorbar_dict = {}
for key, colorbar in zip(['hab', 'pop'], [hab_suit_colorbar, pop_suit_colorbar]):
    fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
    norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list('mymap', colorbar)
    fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=my_cmap), cax=ax, orientation='horizontal',
        label='Risk', ticks=[x[0] for x in colorbar])
    f = io.BytesIO()
    plt.savefig(f, bbox_inches='tight', format='png')
    image = f.getvalue()
    #colorbar_output = widgets.Image(value=image, format='png',)
    colorbar_dict[key] = image
    plt.close()

colorbar_widget = widgets.Image(value=colorbar_dict['hab'], format='png')

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'
# Need to find a way to programmatically get center for diff locations
m = Map(center=map_center, zoom=8, scroll_wheel_zoom=True, layout=Layout(height='600px'))

# Group layers for easier visualization in map control
water_risk_layers = []
population_risk_layers = []
absolute_risk_layers = []
relative_risk_layers = []
# Map layer names to TileLayer representation
checkbox_layers_map = {}
checkbox_legend_map = {}

# Add NDVI layers
# TODO: how do we know which layer have been run?
# Schisto could save a output/layers.txt we could read, which identifies the tile folders...
for season in ['dry', 'wet']:
    # only display layers from the most recent model run
    base_tile_dir = f'ndvi_suit_{season}_tiles'
    if base_tile_dir in layer_name_list:
        ndvi_tile_url = TileLayer(
            url=f'http://localhost:8080/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
            name=f"ndvi {season}",
            attribution="mine", #TODO: do we need anything here?
            min_zoom=1,
            max_zoom=18,
            min_native_zoom=1,
            max_native_zoom=14,)
        water_risk_layers.append(f'ndvi_suit_{season}')
        checkbox_layers_map[f'ndvi_suit_{season}'] = ndvi_tile_url
        checkbox_legend_map[f'ndvi_suit_{season}'] = 'hab'

# Add water temperature layers
for risk_type in ['parasite', 'snail']:
    for season in ['dry', 'wet']:
        # only display layers from the most recent model run
        base_tile_dir = f'{risk_type}_water_temp_suit_{season}_tiles'
        if base_tile_dir in layer_name_list:
            temp_tile_url = TileLayer(
                url=f'http://localhost:8080/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
                name=f"{risk_type} water temp {season}",
                attribution="mine",
                min_zoom=1,
                max_zoom=18,
                min_native_zoom=1,
                max_native_zoom=14,)
            water_risk_layers.append(f'{risk_type}_water_temp_suit_{season}')
            checkbox_layers_map[f'{risk_type}_water_temp_suit_{season}'] = temp_tile_url
            checkbox_legend_map[f'{risk_type}_water_temp_suit_{season}'] = 'hab'

# Add population risk layers
population_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/rural_urbanization_suit_tiles/{z}/{x}/{y}.png',
    name="Population risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
population_risk_layers.append('rural_urbanization_suit')
checkbox_layers_map['rural_urbanization_suit'] = population_risk_tile_url
checkbox_legend_map['rural_urbanization_suit'] = 'pop'

# Add weighted mean risk layer
combined_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/habitat_suit_weighted_mean_tiles/{z}/{x}/{y}.png',
    name="Combined risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
water_risk_layers.append('habitat_suit_weighted_mean')
checkbox_layers_map['habitat_suit_weighted_mean'] = combined_risk_tile_url
checkbox_legend_map['habitat_suit_weighted_mean'] = 'hab'

# Add distance weighted risk (convolution layer)
convolved_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/convolved_hab_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
absolute_risk_layers.append('convolved_hab_risk')
checkbox_layers_map['convolved_hab_risk'] = convolved_risk_tile_url
checkbox_legend_map['convolved_hab_risk'] = 'hab'

# Add normalized distance weighted risk
normalized_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/normalized_convolved_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
relative_risk_layers.append('normalized_convolved_risk')
checkbox_layers_map['normalized_convolved_risk'] = normalized_risk_tile_url
checkbox_legend_map['normalized_convolved_risk'] = 'hab'

# Add relative and absolute risk to population outputs
for risk_type, layer_group in zip(['abs', 'rel'], [absolute_risk_layers, relative_risk_layers]):
    risk_to_pop_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=18,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_{risk_type}'] = risk_to_pop_tile_url
    checkbox_legend_map[f'risk_to_pop_{risk_type}'] = 'pop'
    
    risk_to_pop_count_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_count_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop count {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=18,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_count_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_count_{risk_type}'] = risk_to_pop_count_tile_url
    checkbox_legend_map[f'risk_to_pop_count_{risk_type}'] = 'pop'

out = widgets.Output()
# Add an inital legend control
#leg_dict = {}
#for leg_label, leg_color in pop_suit_legend:
#    leg_dict[leg_label] = leg_color
#legend_control = LegendControl(leg_dict, title="risk", position="bottomright")

@out.capture()
def layer_visible_switch(event):
    widget = event['owner']
    desc_id = widget.description
    print(desc_id)
    layer_tile_url = checkbox_layers_map[desc_id]
    layer_tile_url.visible = event['new']
    cur_layers = m.layers
    layer_order = [cur_layers[0]]
    #print(layer_tile_url)
    #print(layer_tile_url.name)
    for layer in cur_layers[1:]:
        if layer.url != layer_tile_url.url:
            layer_order.append(layer)
    layer_order.append(layer_tile_url)
    print(layer_order)
    m.layers = layer_order
    print(event)
    #print(event['new'])
    
    #layer_leg = checkbox_legend_map[desc_id]
    #print(checkbox_legend_map[desc_id])
    colorbar_widget.value = colorbar_dict[checkbox_legend_map[desc_id]]
    
layer_groups = [water_risk_layers, population_risk_layers, absolute_risk_layers, relative_risk_layers]
layer_group_sections = ["Water risk layers", "Population risk layers", "Absolute risk layers", "Relative risk layers"]
widget_list = []
for group_title, group_list in zip(layer_group_sections, layer_groups):
    group_header_widget = widgets.HTML(description="", value=f'<b>{group_title}<b>')
    checkbox_widget_group = []
    for layer_key in group_list:
        tmp_widget = widgets.Checkbox(
                value=False,
                description=layer_key,
                disabled=False,
                indent=False,
                layout=Layout(padding='0px 0px 0px 10px', margin='0px 0px 0px 0px', width='auto'),
            )
        tmp_widget.observe(layer_visible_switch, names='value')
        checkbox_widget_group.append(tmp_widget)
        checkbox_layers_map[layer_key].visible=False
        m.add(checkbox_layers_map[layer_key])
    
    widget_list.append(widgets.Accordion(children=[widgets.VBox(checkbox_widget_group)], titles=(group_title,)))
    
vbox_widget = widgets.VBox(widget_list, layout=Layout(height='auto', width='auto', overflow_y='auto', padding='0px 0px 0px 0px'))
layers_control_acc = widgets.Accordion(children=[vbox_widget], titles=("Layers",), layout=Layout(max_height='250px', padding='0px 0px 0px 0px'))

widget_control = WidgetControl(widget=layers_control_acc, position="bottomleft")
widget_legend_control = WidgetControl(widget=colorbar_widget, position="bottomright")

onlyfiles = plot_png_list
plot_widgets = []
for plot_image in onlyfiles:
    image = IPython.display.Image(plot_image)
    tmp_widget = widgets.Image(
        value=image.data,
        format='png',
        #width=300,
        #height=400,
    )
    plot_widgets.append(tmp_widget)

plot_grid = widgets.GridBox(plot_widgets, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)", overflow_x='auto'))

map_hbox = widgets.Box([m], layout=Layout(height="70%"))
widgets.VBox([map_hbox, plot_grid])

#%%capture
m.add(widget_control)
# Add an inital legend control
m.add(widget_legend_control)
#m.add(legend_control)

Map(center=[16.152130441193997, -15.959581254692807], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [2]:
out

Output()

In [ ]:
m.controls

In [ ]:
# LEGEND CONTROL
# This method is taken from https://github.com/jupyter-widgets/ipyleaflet/issues/706
import ipywidgets as widgets
import matplotlib.pyplot as plt; import matplotlib as mpl
import io
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
# These hex values map to generic-risk-styles RGB
color_list_ex = [(0, "#ffffb2"), (0.25, "#fecc5c"), (0.5, "#fd8d3c"), (0.75, "#f03b20"), (1.0, "#bd0026")]
my_cmap = mpl.colors.LinearSegmentedColormap.from_list('mymap', color_list_ex)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=my_cmap), cax=ax, orientation='horizontal', label='Some Units', ticks=[0.0, 0.25, 0.5, 0.75, 1.0])
f = io.BytesIO()
plt.savefig(f, bbox_inches='tight', format='png')
image_1 = f.getvalue()
output = widgets.Image(value=image_1, format='png',)
plt.close()
output


In [ ]:
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
pop_suit_legend = [(0, "#f7fbff"), (0.20, "#d1e2f3"), (0.40, "#9ac8e0"), (0.60, "#529dcc"), (0.80, "#1d6cb1"), (1.0, "#08306b")]
#color_list_ex = [(0, "#ffffb2"), (0.25, "#fecc5c"), (0.5, "#fd8d3c"), (0.75, "#f03b20"), (1.0, "#bd0026")]
my_cmap = mpl.colors.LinearSegmentedColormap.from_list('mymap', pop_suit_legend)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=my_cmap), cax=ax, orientation='horizontal', label='Some Units', ticks=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
f = io.BytesIO()
plt.savefig(f, bbox_inches='tight', format='png')
image_2 = f.getvalue()
plt.close()
output.value = image_2
